In [2]:
import pandas as pd
import datetime
import networkx as nx
import plotly.graph_objects as go
import plotly.express as px

In [3]:
file = pd.read_csv("a_disinfectant_retweet_plusorigin_label.csv")

In [4]:
def to_str(v):
    return str(int(float(v)))
def to_int(v):
    return int(float(v))

In [5]:
file = file[file["label"]!=0]

In [6]:
file.label.value_counts()

2    2807
3    1254
1     302
Name: label, dtype: int64

In [7]:
file["all_tweet_id"] = file["tweet_id"].fillna(file["                    tweet_id"]).fillna(file["                tweet_id"])

In [8]:
file.drop(columns=["                    tweet_id","                tweet_id"],inplace = True)

In [8]:
file.columns

Index(['date', 'hts', 'is_quote', 'is_retweet', 'lat', 'likes', 'long',
       'mentions', 'original_tweet_id', 'quote_count', 'quoted_text',
       'replies', 'retweets', 'text', 'tweet_id', 'user', 'label', 'full_text',
       'original_label', 'all_tweet_id'],
      dtype='object')

In [9]:
file_out = file["all_tweet_id"].map(to_str)
file_out.to_excel("rumor_di_tid.xlsx")

In [10]:
file["all_tweet_id"].isnull().value_counts()

False    4363
Name: all_tweet_id, dtype: int64

In [11]:
file["all_tweet_id"] = file["all_tweet_id"].map(to_int)
file["original_tweet_id"] = file["original_tweet_id"].map(to_int)

In [12]:
file.head()

,date,hts,is_quote,is_retweet,lat,likes,long,mentions,original_tweet_id,quote_count,quoted_text,replies,retweets,text,tweet_id,user,label,full_text,original_label,all_tweet_id
1,2020-03-29 00:38:56,NaN,False,True,NaN,0,NaN,NaN,1242529637747625984,0,NaN,0,0,RT @wilsonp911: Remember a couple months ago w...,1.244061e+18,wilsonp911,3,RT @wilsonp911: Remember a couple months ago w...,3,1244061444880162816
5,2020-03-31 14:28:38,Coronavirustruth;covid19zambia,False,True,NaN,0,NaN,NaN,1244994019744235520,0,NaN,0,0,RT @ZambiaAsks: Is spraying alcohol or chlorin...,1.244995e+18,OpenParlyZED,3,RT @ZambiaAsks: Is spraying alcohol or chlorin...,3,1244995022040293376
7,2020-04-01 22:46:44,NaN,False,True,NaN,0,NaN,NaN,1245442852626116608,0,NaN,0,0,RT @billyjack2016: For those idiots who think ...,1.245483e+18,Gamble23Daisy,2,RT @billyjack2016: For those idiots who think ...,2,1245482761525157888
16,2020-04-06 13:53:34,NaN,False,True,NaN,0,NaN,DrGregAllgood;WHO,1247158197103837184,0,NaN,0,0,RT @DrGregAllgood: It was reassuring today to ...,1.247161e+18,BobWeszely,3,RT @DrGregAllgood: It was reassuring today to ...,3,1247160524233674752
23,2020-04-07 15:57:44,NaN,False,True,NaN,0,NaN,NaN,1247549253259382784,0,NaN,0,0,RT @SpeakUp407: For heavenÃ¢ÂÂs sake DONÃ¢Â...,1.247554e+18,stevewoodYYC,2,RT @SpeakUp407: For heavenÃ¢ÂÂs sake DONÃ¢Â...,2,1247554159391326208


In [13]:
file.shape

(4363, 20)

In [14]:
day_list = []
for i in range(file.shape[0]):
    if(file.iloc[i]["date"] == file.iloc[i]["date"]):
        #print(str(file.iloc[i]["date"].split(" ")[0]))
        day = str(file.iloc[i]["date"].split(" ")[0])
        day_list.append(day)
    else:
        day_list.append("0")
file["day"] = day_list

In [15]:
days = file["day"].unique()
dates = [datetime.datetime.strptime(ts, "%Y-%m-%d") for ts in days]
dates.sort()
sorteddates = [datetime.datetime.strftime(ts, "%Y-%m-%d") for ts in dates]

In [16]:
G = nx.Graph()

for i in file.index:
    node1 = int(file[file.index==i]["all_tweet_id"].values[0])
    node1_label = int(file[file.index==i]["label"].values[0])
    node2 = int(file[file.index==i]["original_tweet_id"].values[0])
    node2_label = int(file[file.index==i]["original_label"].values[0])
    date = file[file.index==i]["day"].values[0]
    G.add_node(node1, cat=node1_label, day=date)
    G.add_node(node2, cat=node2_label, day=date)
    G.add_edge(node1, node2)

In [17]:
G.number_of_nodes(), G.number_of_edges()

(4578, 4362)

In [18]:
# store subgraphs in graph_dict = {date:subgraph}
graph_dict = {}
node_day_cum = []
i = 0
for day in sorteddates:
    node_day = [x for x,y in G.nodes(data=True) if y['day']==day]
    node_day_cum += node_day
    G_sub = G.subgraph(node_day_cum)
    graph_dict[day] = G_sub
    i += 1

In [42]:
def showGraph(G, date):
    pos = nx.spring_layout(G)

    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='text',
        mode='lines',
        marker = dict())

    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            # colorscale options
            #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
            #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
            #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
            colorscale='YlGnBu',
            reversescale=True,
            color=[],
            size=10,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line_width=2))

    node_adjacencies = []
    node_text = []
    node_cat = []
    for node, adjacencies in enumerate(G.adjacency()):
        node_adjacencies.append(len(adjacencies[1]))
        node_text.append('\n # of connections: '+str(len(adjacencies[1])))
        node_cat.append(G.nodes[adjacencies[0]]['cat'])

    node_trace.marker.color = node_cat
    node_trace.text = node_text

    fig = go.Figure(data=[edge_trace, node_trace],
                 layout=go.Layout(
                    title=date + "                number of tweets: " + str(G.number_of_nodes()) + "                      "+\
                     str(len([n for n,v in G.nodes(data=True) if v['cat'] == 1])) + " supportive, " + \
                     str(len([n for n,v in G.nodes(data=True) if v['cat'] == 2])) + " denial, " + \
                     str(len([n for n,v in G.nodes(data=True) if v['cat'] == 3])) + " neutral ",
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40),
                    annotations=[ dict(
                        text="",
                        showarrow=False,
                        xref="paper", yref="paper",
                        x=0.005, y=-0.002 ) ],
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )
    fig.show()

In [20]:
graph_dict.keys()

dict_keys(['2020-03-11', '2020-03-12', '2020-03-13', '2020-03-14', '2020-03-15', '2020-03-16', '2020-03-17', '2020-03-18', '2020-03-19', '2020-03-20', '2020-03-22', '2020-03-24', '2020-03-29', '2020-03-31', '2020-04-01', '2020-04-06', '2020-04-07', '2020-04-10', '2020-04-16', '2020-04-17', '2020-04-18', '2020-04-19', '2020-04-23', '2020-04-24', '2020-04-25', '2020-04-26', '2020-04-27', '2020-04-28', '2020-04-29', '2020-04-30', '2020-05-01', '2020-05-02'])

In [44]:
date = '2020-05-02'
H = graph_dict[date]
H_unfrozen = nx.Graph(H)
H_unfrozen.remove_nodes_from([node for node,degree in dict(H.degree()).items() if degree == 0])
showGraph(H_unfrozen, date)

In [49]:
import collections
degree_sequence = sorted([d for n, d in G.degree()], reverse=True)
degreeCount = collections.Counter(degree_sequence)
deg, cnt = zip(*degreeCount.items())
sum_degree = sum(cnt)
#cnt = tuple(t/sum_degree for t in cnt)
deg, cnt

((1387,
  610,
  319,
  310,
  187,
  163,
  123,
  121,
  111,
  90,
  73,
  68,
  51,
  42,
  37,
  36,
  35,
  28,
  27,
  24,
  17,
  16,
  14,
  12,
  11,
  10,
  9,
  8,
  7,
  6,
  5,
  4,
  3,
  2,
  1),
 (1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  3,
  1,
  4,
  4,
  2,
  2,
  3,
  2,
  5,
  8,
  14,
  37,
  4471))

In [47]:
fig = px.scatter(x=deg , y=cnt, log_x=True, log_y=True, labels={"x":"Degree K", "y":"Probability p(k)"}, title="node degree log-log plot for rumor disinfectant") 
fig.show()

In [22]:
day_list = []
cat_1_list = []
cat_2_list = []
cat_3_list = []

for day in graph_dict:
    cat_list = list(nx.get_node_attributes(graph_dict[day], name="cat").values())
    num_1 = cat_list.count(1)
    num_2 = cat_list.count(2)
    num_3 = cat_list.count(3)
    
    day_list.append(day)
    cat_1_list.append(num_1)
    cat_2_list.append(num_2)
    cat_3_list.append(num_3)

fig = go.Figure()
fig.add_trace(go.Scatter(x=day_list, y=cat_1_list,
                    mode='lines',
                    name='supportive',
                    line_shape='spline'))
fig.add_trace(go.Scatter(x=day_list, y=cat_2_list,
                    mode='lines',
                    name='denial',
                    line_shape='spline'))
fig.add_trace(go.Scatter(x=day_list, y=cat_3_list,
                    mode='lines', 
                    name='neutral',
                    line_shape='spline'))

fig.show()

In [48]:
dc = nx.degree_centrality(H)
sort_orders = sorted(dc.items(), key=lambda x: x[1], reverse=True)
sort_orders[0:10]

[(1253861294010220288, 0.3030369237491807),
 (1254186687753674752, 0.13327507100720995),
 (1254037668519972864, 0.06969630762508193),
 (1255242980539146240, 0.06772995411841817),
 (1253540305967566848, 0.04085645619401355),
 (1251213875875217408, 0.035612846842910204),
 (1253610054470811648, 0.026873497924404632),
 (1237126549851320320, 0.026436530478479353),
 (1253490302951682048, 0.02425169324885296),
 (1253868104674947072, 0.019663535066637535)]

In [18]:
file[file["original_tweet_id"]==1253868104674947072]["original_label"]

4282    3
4283    3
4284    3
4285    3
4287    3
       ..
5428    3
5448    3
5584    3
5682    3
5769    3
Name: original_label, Length: 90, dtype: int64